# Evolution of agents - MCP

MCP (Model Context Protocol) is an open spec that standardizes how AI clients (agents, IDEs, chat apps) discover and call external tools and access resources. It uses a client–server model where servers expose capabilities (tools/functions, prompts, resources like files/DBs) over JSON-RPC transports (e.g., stdio, WebSocket). The goal is portability and safety: tools are reusable across different LLM runtimes with consistent schemas, auth, and streaming I/O, while remaining sandboxed. In practice, you run an MCP server for a service and any MCP-compatible client can list its capabilities and invoke them.

<img src="./images/agent_actions.png" alt="Agent with Actions" style="max-height: 300px;" />


In [ ]:
# Example: Inference using Semantic Kernel
from semantic_kernel import Kernel
import os
from setup import (
    get_project_client,
    create_agent,
    create_weather_openapi_tool,
    get_connection_by_name,
    test_agent,
)
from AzureStandardLogicAppTool import create_logic_app_tools

client = await get_project_client()

logic_app_tools = create_logic_app_tools(
    logic_app_subscription_id=os.environ.get("LOGIC_APP_SUBSCRIPTION_ID"),
    logic_app_resource_group=os.environ.get("LOGIC_APP_RESOURCE_GROUP"),
    logic_app_name=os.environ.get("LOGIC_APP_NAME"),
    foundry_subscription_id=os.environ.get("AZURE_AI_FOUNDRY_SUBSCRIPTION_ID"),
    foundry_resource_group=os.environ.get("AZURE_AI_FOUNDRY_RESOURCE_GROUP"),
    foundry_foundry_name=os.environ.get("AZURE_AI_FOUNDRY_NAME"),
    foundry_project_name=os.environ.get("AZURE_AI_FOUNDRY_PROJECT_NAME"),
)
tool_definitions = []
for tool in logic_app_tools:
    tool_definitions = tool_definitions + tool.definitions

kernel = Kernel()

office_agent = await create_agent(
    agent_name="Office365-agent",
    agent_instructions="You are a helpful assistant. You use Office 365 tools to send emails, schedule events and check calendars.",
    client=client,
    kernel=kernel,
    tools=[def_ for tool in logic_app_tools for def_ in tool.definitions],
)

weather_tool = create_weather_openapi_tool()

weather_agent = await create_agent(
    agent_name="Weather-agent",
    agent_instructions="You are a helpful assistant. You use weather tools to provide weather information.",
    client=client,
    kernel=kernel,
    tools=weather_tool.definitions,
)

playwright_agent = await create_agent(
    agent_name="Playwright-agent",
    agent_instructions="You are a helpful assistant. You use Playwright tools to interact with web pages.",
    client=client,
    kernel=kernel,
    tools=[
        {
            "type": "browser_automation",
            "browser_automation": {
                "connection": {"id": await get_connection_by_name(client, "Playwright")}
            },
        }
    ],
)

thread = None
user_input = (
    "go to https://reindeerromp5k.com/ and check dates, prices and details for the race"
)
thread = await test_agent(client, playwright_agent, user_input, thread)